In [ ]:
from data_pre_processing import *
from pair_finder import *
from back_tester import *
import itertools
import pandas as pd
from statsmodels.tsa.stattools import coint
from itertools import combinations



/Users/alex_1/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


# Load data 
---

In [2]:
# prices_XRP = pd.read_csv('binance_data/top_100_tickers/2024/1m/XRPUSDT_2024_1m.csv', index_col=0, parse_dates=True)

In [3]:
# #Check if there are any cases where the close price does not change
# prices_XRP_dff = prices_XRP['Close'].diff()

### Resampling

In [4]:
# resample_data('binance_data/Wrapped BTC/2024/1m', '30T', 'binance_data/Wrapped BTC/2024/30m2')

### Merge CSVS

In [5]:
# merge_ohlc_closing_prices("binance_data/top_100_tickers/2024/1m")

In [6]:
# #Manual adjustments
# import pandas as pd

# # Load dataset (replace with actual file path if loading from a CSV file)
# df = pd.read_csv("binance_data/Staked_ETH_Bybit/BYBIT_ETHUSDT, 30_1dfe3.csv")  # Replace with actual file path if needed

# # Convert 'time' column to datetime format and rename it
# df['Open Time'] = pd.to_datetime(df['time'], unit='s')

# # Drop the old 'time' column if needed
# df = df.drop(columns=['time'])

# #Rename close to Close
# df = df.rename(columns={'close': 'Close'})

# # Reorder columns to make 'Open Time' the first column
# column_order = ['Open Time'] + [col for col in df.columns if col != 'Open Time']
# df = df[column_order]

# cutoff_time = pd.Timestamp("2024-12-31 23:30:00")

# # Filter the dataframe to only include rows up to the cutoff time
# df = df[df['Open Time'] <= cutoff_time]


# df.to_csv("binance_data/Staked_ETH_Bybit/BYBIT_ETHUSDT, 30_1dfe3.csv", index=False)


In [7]:
# df = pd.read_csv("binance_data/Staked_ETH_Bybit/BYBIT_STETHUSDT, 30_fd7e3.csv")  # Replace with actual file path if needed
# df

* ## Closing price data
---

In [8]:

#BTC and WBTC
#-------------------------------------------
#prices = pd.read_csv("binance_data/Wrapped BTC/2024/30m/merged_closing_prices.csv", index_col=0, parse_dates=True)


# #SOL and BNSOL
# #-------------------------------------------
# prices = pd.read_csv('binance_data/SOL_and_BNSOL/2025/1H/merged_closing_prices.csv', index_col=0, parse_dates=True)

# #ETH and RPL
# #-------------------------------------------
#prices = pd.read_csv('binance_data/ETH_and_RPL/2024/1m/merged_closing_prices.csv', index_col=0, parse_dates=True)

# #ETH and WBETH
# #-------------------------------------------
#prices = pd.read_csv('binance_data/ETH_and_WBETH/2024/1m/merged_closing_prices.csv', index_col=0, parse_dates=True)

#Top 100
#prices = pd.read_csv('binance_data/top_100_tickers/2024/1m/merged_closing_prices.csv', index_col=0, parse_dates=True)

#Tardis 5min trades data for 60 coins
prices = pd.read_csv('tardis_data/final_in_sample_dataset/final_in_sample_dataset_5min_2024.csv', index_col=0, parse_dates=True)

In [9]:
# #Filter prices for XRP and ADA
#prices = prices[['XRP/USDT_2024_1m', 'ADA/USDT_2024_1m']]
#prices = prices[['ADA/USDT_2024_1m', 'XRP/USDT_2024_1m']]

# #Filter by DOT and XTZ
# prices = prices[['DOT/USDT_2024_1m', 'XTZ/USDT_2024_1m']]

In [10]:
# score, pvalue, _ = coint(prices[prices.columns[1]],prices[prices.columns[0]])
# print(f"Score: {score}")
# print(f"P-value: {pvalue}")


In [11]:
# prices = prices[prices.diff().ne(0).all(axis=1)]

In [12]:
# count_unchanged_rows = (prices.diff().eq(0).all(axis=1)).sum()
# print("Number of fully unchanged rows:", count_unchanged_rows)

* ## Order book data

In [13]:
# #BTC and WBTC
# #-------------------------------------------
# prices = pd.read_csv('order_book_data/merged_data/1min/btc_wbtc_combined_1m.csv', index_col=0, parse_dates=True)

In [14]:
# # Only show columns = btc_mid_price and wbtc_mid_price
# prices = prices[['btc_mid_price', 'wbtc_mid_price']]

In [15]:
# #Plot the closing prices of the merged data using different y axis

# crypto_1 = prices.columns[0]
# crypto_2 = prices.columns[1]

# fig, ax1 = plt.subplots()

# #Set size of the plot
# fig.set_size_inches(12, 8)

# color = 'tab:red'
# ax1.set_xlabel('Time')
# ax1.set_ylabel(f'{crypto_1} Closing Price', color=color)
# ax1.plot(prices.index, prices[crypto_1], color=color)
# ax1.tick_params(axis='y', labelcolor=color)

# ax2 = ax1.twinx()
# color = 'tab:blue'
# ax2.set_ylabel(f'{crypto_2} Closing Price', color=color)
# ax2.plot(prices.index, prices[crypto_2], color=color)
# ax2.tick_params(axis='y', labelcolor=color)

# #Add a title to the plot
# plt.title(f'{crypto_1} and {crypto_2} Closing Prices')







In [16]:
# #Calculate correlation matrix
# correlation_matrix = prices.corr()
# correlation_matrix

# Test for pairs
---

* ## Apply correlation filter

In [17]:
# high_corr_pairs = [('ETC/USDT_2024_1m',
#   'ETH/USDT_2024_1m',
#   np.float64(0.1506849315068493)
#  )]

In [20]:

# # Pre-filter: Calculate correlation matrix and select pairs with high correlation
# corr_threshold = 0.8
# # corr_matrix, high_corr_pairs = filter_high_correlation_pairs(prices, threshold=corr_threshold)
# print(f"\nPairs with correlation >= {corr_threshold}:")
# for pair in high_corr_pairs:
#     print(f"{pair[0]} & {pair[1]}: correlation = {pair[2]:.4f}")

# # Cointegration test
# cointegrated_pairs, pvalue_matrix, residuals_df = find_cointegrated_pairs(prices, high_corr_pairs,significance=0.05)
# #cointegrated_pairs, window_results = find_cointegrated_pairs_windows(prices, high_corr_pairs, significance=0.05, window_size=720, min_pass_fraction=0.5)


# Filter prices dataframe into communities
---

In [18]:
communities_df = pd.read_csv('results/community_detection/communities_2024.csv', parse_dates=True)

#Get the tickers in communities df where the community is 1
community_1_tickers = communities_df[communities_df['Community'] == 'Community 1']['Ticker'].tolist()
community_2_tickers = communities_df[communities_df['Community'] == 'Community 2']['Ticker'].tolist()

#Filter prices for community 1
prices_community_1 = prices[community_1_tickers]
#Filter prices for community 2
prices_community_2 = prices[community_2_tickers]

# Find pairs
---

* ## Communnity 1 pairs

In [48]:

# #Community 1 pairs
# #----

# # Store results here
# cointegration_results_community_1 = []

# # Loop through all unique pairs of columns
# for col1, col2 in combinations(prices_community_1.columns, 2): #combinations(iterable, r):This function generates all possible combinations of r elements from the given iterable without repetition.
#     score, pvalue, _ = coint(prices_community_1[col1], prices_community_1[col2])

#     if(pvalue < 0.05):
#         print(f"Cointegrated pair: {col1} & {col2}, p-value: {pvalue:.4f}, score: {score:.4f}")

#         cointegration_results_community_1.append({
#             'pair': f"{col1} & {col2}",
#             'pvalue': pvalue,
#             'score': score
#         })

# # Convert results to a DataFrame
# coint_df_community_1 = pd.DataFrame(cointegration_results_community_1)

# # Sort by lowest p-value (strongest evidence of cointegration)
# coint_df_community_1 = coint_df_community_1.sort_values('pvalue')

# #Save results to CSV
# coint_df_community_1.to_csv('results/pairs/pairs_community_1.csv', index=False)


Cointegrated pair: SUIUSDT_2024_5m & IOTAUSDT_2024_5m, p-value: 0.0234, score: -3.6161
Cointegrated pair: SUIUSDT_2024_5m & EGLDUSDT_2024_5m, p-value: 0.0098, score: -3.9029
Cointegrated pair: FETUSDT_2024_5m & GALAUSDT_2024_5m, p-value: 0.0435, score: -3.3895
Cointegrated pair: FETUSDT_2024_5m & THETAUSDT_2024_5m, p-value: 0.0076, score: -3.9809
Cointegrated pair: JTOUSDT_2024_5m & NEARUSDT_2024_5m, p-value: 0.0084, score: -3.9503
Cointegrated pair: AXSUSDT_2024_5m & EOSUSDT_2024_5m, p-value: 0.0218, score: -3.6400
Cointegrated pair: AXSUSDT_2024_5m & IOTAUSDT_2024_5m, p-value: 0.0075, score: -3.9854
Cointegrated pair: AXSUSDT_2024_5m & FLOWUSDT_2024_5m, p-value: 0.0167, score: -3.7316
Cointegrated pair: AXSUSDT_2024_5m & MANAUSDT_2024_5m, p-value: 0.0006, score: -4.6928
Cointegrated pair: AXSUSDT_2024_5m & APTUSDT_2024_5m, p-value: 0.0442, score: -3.3834
Cointegrated pair: AXSUSDT_2024_5m & XTZUSDT_2024_5m, p-value: 0.0149, score: -3.7681
Cointegrated pair: AXSUSDT_2024_5m & SANDUSDT

* ## Communnity 2 pairs

In [46]:

# #Community 2 pairs
# #----

# # Store results here
# cointegration_results_community_2 = []

# # Loop through all unique pairs of columns
# for col1, col2 in combinations(prices_community_2.columns, 2):
#     score, pvalue, _ = coint(prices_community_2[col1], prices_community_2[col2])

#     if(pvalue < 0.05):
#         print(f"Cointegrated pair: {col1} & {col2}, p-value: {pvalue:.4f}, score: {score:.4f}")

#         cointegration_results_community_2.append({
#             'pair': f"{col1} & {col2}",
#             'pvalue': pvalue,
#             'score': score
#         })


# # Convert results to a DataFrame
# coint_df_community_2 = pd.DataFrame(cointegration_results_community_2)

# # Sort by lowest p-value (strongest evidence of cointegration)
# coint_df_community_2 = coint_df_community_2.sort_values('pvalue')

# #Save results to CSV
# coint_df_community_2.to_csv('results/pairs/pairs_community_2.csv', index=False)


Cointegrated pair: SHIBUSDT_2024_5m & LTCUSDT_2024_5m, p-value: 0.0252, score: -3.5900
Cointegrated pair: DOGEUSDT_2024_5m & BCHUSDT_2024_5m, p-value: 0.0078, score: -3.9746
Cointegrated pair: DOGEUSDT_2024_5m & LTCUSDT_2024_5m, p-value: 0.0009, score: -4.5694
Cointegrated pair: QNTUSDT_2024_5m & XRPUSDT_2024_5m, p-value: 0.0027, score: -4.2816
Cointegrated pair: QNTUSDT_2024_5m & XLMUSDT_2024_5m, p-value: 0.0015, score: -4.4538
Cointegrated pair: XRPUSDT_2024_5m & ADAUSDT_2024_5m, p-value: 0.0030, score: -4.2512
Cointegrated pair: XRPUSDT_2024_5m & AAVEUSDT_2024_5m, p-value: 0.0012, score: -4.4976
Cointegrated pair: XRPUSDT_2024_5m & DOTUSDT_2024_5m, p-value: 0.0001, score: -5.0393
Cointegrated pair: XRPUSDT_2024_5m & XLMUSDT_2024_5m, p-value: 0.0014, score: -4.4611
Cointegrated pair: BCHUSDT_2024_5m & LTCUSDT_2024_5m, p-value: 0.0484, score: -3.3484
Cointegrated pair: BCHUSDT_2024_5m & MKRUSDT_2024_5m, p-value: 0.0426, score: -3.3974
Cointegrated pair: LTCUSDT_2024_5m & MKRUSDT_2024_

In [31]:
prices_community_2.columns

Index(['SHIBUSDT_2024_5m', 'DOGEUSDT_2024_5m', 'QNTUSDT_2024_5m',
       'BONKUSDT_2024_5m', 'XRPUSDT_2024_5m', 'BCHUSDT_2024_5m',
       'ADAUSDT_2024_5m', 'DEXEUSDT_2024_5m', 'LTCUSDT_2024_5m',
       'AAVEUSDT_2024_5m', 'MKRUSDT_2024_5m', 'ETCUSDT_2024_5m',
       'PEPEUSDT_2024_5m', 'TRXUSDT_2024_5m', 'VETUSDT_2024_5m',
       'NEOUSDT_2024_5m', 'NEXOUSDT_2024_5m', 'BTCUSDT_2024_5m',
       'DOTUSDT_2024_5m', 'PAXGUSDT_2024_5m', 'ETHUSDT_2024_5m',
       'BNBUSDT_2024_5m', 'XLMUSDT_2024_5m'],
      dtype='object')

* ## Cross community pairs


In [32]:

#Cross community pairs
#----

# Store results here
cointegration_results_cross_community = []

for community_1_col in prices_community_1.columns:
    for community_2_col in prices_community_2.columns:
        score, pvalue, _ = coint(prices_community_1[community_1_col], prices_community_2[community_2_col])

        if(pvalue < 0.05):
            print(f"Cointegrated pair: {community_1_col} & {community_2_col}, p-value: {pvalue:.4f}, score: {score:.4f}")

            cointegration_results_cross_community.append({
                'pair': f"{community_1_col} & {community_2_col}",
                'pvalue': pvalue,
                'score': score
            })


# Convert results to a DataFrame
coint_df_cross_communities = pd.DataFrame(cointegration_results_cross_community)

# Sort by lowest p-value (strongest evidence of cointegration)
coint_df_cross_communities = coint_df_cross_communities.sort_values('pvalue')

#Save results to CSV
coint_df_cross_communities.to_csv('results/pairs/pairs_cross_communities.csv', index=False)


Cointegrated pair: SUIUSDT_2024_5m & QNTUSDT_2024_5m, p-value: 0.0298, score: -3.5308
Cointegrated pair: FETUSDT_2024_5m & SHIBUSDT_2024_5m, p-value: 0.0017, score: -4.4069
Cointegrated pair: FETUSDT_2024_5m & DOGEUSDT_2024_5m, p-value: 0.0019, score: -4.3811
Cointegrated pair: FETUSDT_2024_5m & LTCUSDT_2024_5m, p-value: 0.0042, score: -4.1628
Cointegrated pair: JTOUSDT_2024_5m & BCHUSDT_2024_5m, p-value: 0.0254, score: -3.5871
Cointegrated pair: AXSUSDT_2024_5m & XRPUSDT_2024_5m, p-value: 0.0000, score: -5.3392
Cointegrated pair: AXSUSDT_2024_5m & AAVEUSDT_2024_5m, p-value: 0.0320, score: -3.5040
Cointegrated pair: AXSUSDT_2024_5m & DOTUSDT_2024_5m, p-value: 0.0001, score: -5.1476
Cointegrated pair: AXSUSDT_2024_5m & XLMUSDT_2024_5m, p-value: 0.0229, score: -3.6230
Cointegrated pair: GALAUSDT_2024_5m & LTCUSDT_2024_5m, p-value: 0.0009, score: -4.5807
Cointegrated pair: EOSUSDT_2024_5m & ETCUSDT_2024_5m, p-value: 0.0396, score: -3.4250
Cointegrated pair: IOTAUSDT_2024_5m & XRPUSDT_2024

In [34]:
# #Fliter pairs with p-value < 0.05
# cointegrated_pairs = coint_df_community_2[coint_df_community_2['pvalue'] < 0.05]
# cointegrated_pairs.to_csv('cointegrated_pairs.csv', index=False)
# cointegrated_pairs

,pair,pvalue,score
9,BCH/USDT_2024_1m & DOGE/USDT_2024_1m,0.004478,-4.140725
37,GALA/USDT_2024_1m & GRT/USDT_2024_1m,0.005658,-4.071777
24,ETC/USDT_2024_1m & MANA/USDT_2024_1m,0.036769,-3.453269
28,ETC/USDT_2024_1m & GRT/USDT_2024_1m,0.039311,-3.428222
26,ETC/USDT_2024_1m & WLD/USDT_2024_1m,0.042074,-3.402499
25,ETC/USDT_2024_1m & GALA/USDT_2024_1m,0.046570,-3.363491


In [21]:
# from statsmodels.tsa.stattools import coint, adfuller

# #coin_t, pvalue, crit_value = coint(prices['XRP/USDT_2024_1m'], prices['ADA/USDT_2024_1m'])
# coin_t, pvalue, crit_value = coint(prices['DOT/USDT_2024_1m'], prices['XTZ/USDT_2024_1m'])


# if(pvalue < 0.05):
#     print("Cointegrated")
# else:
#     print("Not Cointegrated")

# print(f"Cointegration test p-value: {pvalue:.4f}")
# print(f"Cointegration test critical value: {crit_value}")

Cointegrated
Cointegration test p-value: 0.0202
Cointegration test critical value: [-3.89652106 -3.33617522 -3.04448139]


In [22]:
# high_corr_pairs = []

* ## Test cointegration in windows

In [23]:
# #Params
# window_size = 1440 # 1 day windows 
# min_pass_fraction = 0
# significance = 0.05

# #Find cointegrated pairs
# cointegrated_pairs, window_results, pair_corr, pass_fraction, avg_pvalue = find_cointegrated_pairs_windows(prices, high_corr_pairs, significance, window_size, min_pass_fraction)


Cointegrated pairs (across windows):
DOT/USDT_2024_1m & XTZ/USDT_2024_1m: pass fraction=0.13, avg p-value=0.4305, correlation=0.9469772978525677


In [24]:
# cointegrated_pairs

[('DOT/USDT_2024_1m',
  'XTZ/USDT_2024_1m',
  np.float64(0.12903225806451613),
  np.float64(0.43054026663820555),
  0.0)]

In [22]:
# #Window results
# windows_list = window_results[('BTC/USDT_2024_30m', 'WBTC/USDT_2024_30m')]
# len(windows_list)

In [21]:
# #Plot windows

# windows = split_price_series_into_windows(prices, window_size)


# crypto_1 = prices.columns[0]
# crypto_2 = prices.columns[1]

# #For ICP/USDT_2024_1h', 'ADA/USDT_2024_1h
# plot_spread_in_windows(crypto_1, crypto_2, windows, window_results, significance=0.05) #These spreads look like they are standardised, but they are not. They are centered around 0 due to the inclusion of the intercept term in the cointegration test. The spread is still in the original units of the data.
# #plot_spread_in_windows('S1', 'S2', windows, window_results, significance=0.05)

# #Check number of windows
# #-----------------------
# window_results.values()

# #Count number of values in window_results
# num_windows = 0
# for key in window_results.keys():
#     print(key)
#     num_windows += len(window_results[key])

# print(f"Number of windows: {num_windows}")

* ### Check resiuals

In [15]:
# #If you suspect daily intraday patterns, using 24 lags is a natural choice: it checks each hour in a 24-hour cycle for dependence.
# #If your sample is large enough (covering many days/weeks), 24 lags is typically enough to detect standard intraday correlation
# analyze_residuals(residuals_df, lags = 24)

# Validation of cointegration methods using data designed for cointegration
---

In [16]:
# import numpy as np
# import pandas as pd

# def generate_cointegrated_data(
#     n=720,  # number of data points
#     alpha=5.0,
#     beta=1.5,
#     phi=0.8, #manually set to 0.8 so that the spread is stationary (no unit root)
#     seed=42
# ):
#     """
#     Generate two cointegrated series: S1 (random walk) and S2 (linear function of S1 + stationary AR(1) noise).
    
#     Parameters:
#         n (int): Number of observations (e.g. 720 for ~1 month of hourly data).
#         alpha (float): Intercept term for S2.
#         beta (float): S2 coefficient for S1.
#         phi (float): AR(1) coefficient for the noise in S2 - beta*S1. Must be <1 in abs value for stationarity.
#         seed (int): Random seed for reproducibility.

#     Returns:
#         pd.DataFrame with columns ['S1', 'S2'] representing the two cointegrated time series.
#     """
#     rng = np.random.default_rng(seed)
    
#     # 1) Generate S1 as a random walk
#     #   S1_t = S1_{t-1} + e1_t
#     e1 = rng.normal(loc=0.0, scale=1.0, size=n)
#     S1 = np.cumsum(e1)  # cumsum => random walk
    
#     # 2) Generate a stationary AR(1) noise for the spread
#     #   ARnoise_t = phi * ARnoise_{t-1} + e2_t
#     e2 = rng.normal(loc=0.0, scale=1.0, size=n)
#     ARnoise = np.zeros(n)
#     for t in range(1, n):
#         ARnoise[t] = phi * ARnoise[t-1] + e2[t]
#     # ARnoise is stationary if |phi| < 1
    
#     # 3) Define S2 = alpha + beta*S1 + AR(1) noise
#     S2 = alpha + beta*S1 + ARnoise
    
#     # Put them in a DataFrame
#     df = pd.DataFrame({'S1': S1, 'S2': S2})
#     return df

# # Example usage
# if __name__ == "__main__":
#     df_cointegrated = generate_cointegrated_data(n=720)
#     print(df_cointegrated.head(10))


In [17]:
# # Suppose df has columns: 'S1', 'S2'
# alpha = 5.0
# beta = 1.5

# # Calculate the spread: S2 - (alpha + beta * S1)
# df_cointegrated['spread'] = df_cointegrated['S2'] - alpha - beta * df_cointegrated['S1']

# # Plot
# plt.figure(figsize=(10,6))
# plt.plot(df_cointegrated['spread'], label='Spread = S2 - alpha - beta*S1')
# plt.axhline(df_cointegrated['spread'].mean(), color='red', linestyle='--', label='Mean')
# plt.title('Spread Over Time')
# plt.legend()
# plt.show()


In [18]:
# cointegrated_pairs, window_results = find_cointegrated_pairs_windows(df_cointegrated, high_corr_pairs, significance=0.05, window_size=240, min_pass_fraction=0.5)

In [19]:
# window_results